## Step 0 - Import Modules

In [44]:
import json
import pandas as pd
import os
from typing import List

## Step 1 - Hello, Data!

In [45]:
# Step 1 – Hello, Data!

# Load the raw CSV (replace with your actual file path)
df = pd.read_csv("data/synthetic_ecommerce_data.csv")

# Display the first 3 rows
df.head(3)

,date,customer_id,product,price,quantity,coupon_code,shipping_city,country,total_price
0,2024-10-19,8be304df-4697-4f40-9f57-5d849fe3947c,Smartphone,699.99,3,NaN,Calgary,Canada,2099.97
1,2025-01-04,124f4ee9-b31b-4c15-a22c-21fd2a263098,Monitor,249.99,2,20OFF,Ottawa,Canada,499.98
2,2024-08-18,fcaac143-fab7-494a-b400-cf3e2f6d6d07,Laptop,999.99,6,NO_CODE,Leeds,UK,5999.94


## Step 2 – Pick the Right Container

For transaction records, a class is most appropriate because it allows us to encapsulate data and behavior (e.g., cleaning, calculating total revenue/profit) in a single unit. Namedtuples are immutable and lack methods; dictionaries are flexible but offer no structure or validation.

In [46]:
# Step 3 – Transaction Class and OO Data Structure
class Transaction:
    def __init__(self, date, customer_id, product, price, quantity,
                 coupon_code, shipping_city, country, total_price):
        self.date = date
        self.customer_id = customer_id
        self.product = product

        self.price = float(price)
        self.quantity = int(quantity)
        self.coupon_code = coupon_code or "NO_CODE"
        self.shipping_city = shipping_city

        self.country = country
        self.total_price = float(total_price)

    def total(self):
        return self.price * self.quantity

    def clean(self):
        if isinstance(self.product, str):
            self.product = self.product.strip().title()

        if isinstance(self.shipping_city, str):
            self.shipping_city = self.shipping_city.strip().title()

        if isinstance(self.country, str):
            self.country = self.country.strip().title()
            
        if not self.coupon_code:
            self.coupon_code = "NO_CODE"



## Step 4 - Bulk Loader

In [47]:
def load_transactions(df: pd.DataFrame) -> List[Transaction]:
    transactions = []
    for i in range(len(df)):
        row = df.iloc[i]
        transaction = Transaction(
            row["date"],
            row["customer_id"],
            row["product"],
            row["price"],
            row["quantity"],
            row["coupon_code"],
            row["shipping_city"],
            row["country"],
            row["total_price"]
        )
        transactions.append(transaction)
    return transactions

# Load all transactions
transactions = load_transactions(df)

## Step 5 - Quick Profiling

In [48]:
# Step 5 – Quick Profiling

# Basic stats on price and quantity
prices = [t.price for t in transactions]
quantities = [t.quantity for t in transactions]
total_prices = [t.total_price for t in transactions]
cities = {t.shipping_city for t in transactions}
countries = {t.country for t in transactions}

print("Min price:", min(prices))
print("Mean price:", round(sum(prices) / len(prices), 2))
print("Max price:", max(prices))

print("Min quantity:", min(quantities))
print("Max quantity:", max(quantities))

print("Total revenue (calculated):", round(sum(total_prices), 2))

print("Unique cities:", len(cities))
print("Unique countries:", len(countries))


Min price: 89.99
Mean price: 442.83
Max price: 999.99
Min quantity: 1
Max quantity: 10
Total revenue (calculated): 1375038.97
Unique cities: 15
Unique countries: 3


## Step 6 - Spot the Grime


In [49]:
# Step 6 – Spot the Grime

# Show rows with missing or clearly malformed data
dirty_rows = df[df.isna().any(axis=1)]
dirty_rows.head(5)

,date,customer_id,product,price,quantity,coupon_code,shipping_city,country,total_price
0,2024-10-19,8be304df-4697-4f40-9f57-5d849fe3947c,Smartphone,699.99,3,NaN,Calgary,Canada,2099.97
3,2025-01-03,bf0df2a4-d230-4113-8150-e0f6e8688588,Smartphone,699.99,8,NaN,Manchester,UK,5599.92
9,2025-04-19,798de1c5-1027-46fa-9c35-c3718a8810f1,Smartphone,699.99,4,NaN,London,UK,2799.96
19,2024-08-07,9ff80ba1-ebe6-483a-bd7f-888b0a38f6d7,Smartphone,699.99,1,NaN,Glasgow,UK,699.99
20,2025-04-21,4f239858-8316-4c9f-8718-62cc20396dc4,Headphones,149.99,6,NaN,Manchester,UK,899.94


## Step 7 - Cleaning Rules

In [50]:
# Step 7 – Cleaning Rules

# Helper function to check if a coupon code is missing or blank
def is_empty_coupon(code):
    return not isinstance(code, str) or code.strip() == ""

# --- Check for issues before cleaning ---
unclean_country_count = sum(t.country != t.country.title() for t in transactions)
unclean_city_count = sum(t.shipping_city != t.shipping_city.title() for t in transactions)
empty_coupon_count = sum(is_empty_coupon(t.coupon_code) for t in transactions)

print("Before cleaning:")
print("  Countries with incorrect capitalization:", unclean_country_count)
print("  Shipping cities with incorrect capitalization:", unclean_city_count)
print("  Missing or blank coupon codes:", empty_coupon_count)

# --- Apply cleaning to each transaction ---
for transaction in transactions:
    transaction.clean()

# --- Recheck after cleaning ---
still_unclean_country_count = sum(t.country != t.country.title() for t in transactions)
still_unclean_city_count = sum(t.shipping_city != t.shipping_city.title() for t in transactions)
no_code_coupon_count = sum(t.coupon_code == "NO_CODE" for t in transactions)

print("\nAfter cleaning:")
print("  Countries still not titlecased:", still_unclean_country_count)
print("  Shipping cities still not titlecased:", still_unclean_city_count)
print("  Coupon codes now set to 'NO_CODE':", no_code_coupon_count)


Before cleaning:
  Countries with incorrect capitalization: 371
  Shipping cities with incorrect capitalization: 0
  Missing or blank coupon codes: 121

After cleaning:
  Countries still not titlecased: 0
  Shipping cities still not titlecased: 0
  Coupon codes now set to 'NO_CODE': 104


## Step 8 - Transformations

In [51]:
# Step 8 – Transformations

# Define how discounts are derived from coupon codes
def parse_discount(code: str) -> float:
    if not isinstance(code, str):
        return 0.0
    code = code.strip().upper()
    if code == "10OFF":
        return 0.10
    elif code == "20OFF":
        return 0.20
    elif code == "FREESHIP":
        return 0.05  # e.g. 5% as shipping compensation
    return 0.0

# Apply transformation to each transaction
for t in transactions:
    t.discount = parse_discount(t.coupon_code)
    t.total_after_discount = round(t.total() * (1 - t.discount), 2)

# Show a few transformed examples
for t in transactions[:5]:
    print(f"{t.coupon_code} ➝ {t.discount*100:.0f}% off ➝ Final total: ${t.total_after_discount}")


nan ➝ 0% off ➝ Final total: $2099.97
20OFF ➝ 20% off ➝ Final total: $399.98
NO_CODE ➝ 0% off ➝ Final total: $5999.94
nan ➝ 0% off ➝ Final total: $5599.92
10OFF ➝ 10% off ➝ Final total: $1574.94


## Step 9 - Feature Engineering

In [52]:
from datetime import datetime

# Step 9 – Feature Engineering

# Today's date for calculating days since purchase
today = pd.Timestamp(datetime.today().date())

# Enrich each transaction with new features
for transaction in transactions:
    # High-value if total before discount > $1000
    transaction.is_high_value = transaction.total() > 1000

    # Deterministic shipping delay based on day of month (e.g., 2–6 days)
    try:
        order_date = pd.to_datetime(transaction.date)
        transaction.shipping_delay = 2 + (order_date.day % 5)

        # Days since purchase (difference from today)
        transaction.days_since_purchase = (today - order_date).days
    except Exception:
        transaction.shipping_delay = None
        transaction.days_since_purchase = None

# Show a few enriched examples
print("Sample transactions with new features:")
for tx in transactions[:5]:
    print(f"- Total: ${tx.total():.2f} | High Value: {tx.is_high_value} | "
          f"Shipping Delay: {tx.shipping_delay} days | "
          f"Days Since Purchase: {tx.days_since_purchase}")


Sample transactions with new features:
- Total: $2099.97 | High Value: True | Shipping Delay: 6 days | Days Since Purchase: 220
- Total: $499.98 | High Value: False | Shipping Delay: 6 days | Days Since Purchase: 143
- Total: $5999.94 | High Value: True | Shipping Delay: 5 days | Days Since Purchase: 282
- Total: $5599.92 | High Value: True | Shipping Delay: 5 days | Days Since Purchase: 144
- Total: $1749.93 | High Value: True | Shipping Delay: 6 days | Days Since Purchase: 13


## Step 10 - Mini-Aggregation

In [53]:
# Step 10 – Mini-Aggregation: Revenue by shipping_city
from collections import defaultdict

revenue_by_city = defaultdict(float)
for t in transactions:
    revenue_by_city[t.shipping_city] += t.total_after_discount

# Sort and display top 5 cities
sorted_revenue = sorted(revenue_by_city.items(), key=lambda x: x[1], reverse=True)
for city, revenue in sorted_revenue[:5]:
    print(f"{city}: ${revenue:.2f}")


Manchester: $111755.44
Chicago: $111740.35
Los Angeles: $111262.08
Houston: $105837.12
Leeds: $93495.50


## Step 11 - Serialization Checkpoint

In [ ]:


# Convert Transaction objects to dictionaries for export
def transaction_to_dict(t):
    return {
        "date": t.date,
        "customer_id": t.customer_id,
        "product": t.product,
        "price": t.price,
        "quantity": t.quantity,
        "coupon_code": t.coupon_code,
        "shipping_city": t.shipping_city,
        "country": t.country,
        "total_price": t.total_price,
        "discount": t.discount,
        "total_after_discount": t.total_after_discount,
        "is_high_value": t.is_high_value,
        "shipping_delay": t.shipping_delay,
    }

records = [transaction_to_dict(t) for t in transactions]


##### I used https://www.datacamp.com/tutorial/apache-parquet to understand how to do this as i have never used parquet before

# Save as JSON
os.makedirs("exports", exist_ok=True)
with open("exports/cleaned_data.json", "w") as f:
    json.dump(records, f, indent=2)

# Save as Parquet

df_export = pd.DataFrame(records)
df_export.to_parquet("exports/cleaned_data.parquet", index=False)

print("Saved as cleaned_data.json and cleaned_data.parquet")


Saved as cleaned_data.json and cleaned_data.parquet


## Step 12 - Soft Interview Reflection

Using OOP, each transaction was modeled as a class object with built-in methods for cleaning and calculations. This kept the code modular, clear, and consistent, making it easier to process and maintain the data pipeline. 

## Data-Dictionary Section

In [55]:
product_catalog = pd.read_csv("data/secondary_product_catalog.csv")

# Descriptions for main dataset fields
field_info = {
    "date": "Order date of the transaction",
    "customer_id": "Unique identifier for each customer or order",
    "product": "Name of the product sold",
    "price": "Unit price of the product",
    "quantity": "Number of units sold in the transaction",
    "coupon_code": "Discount or promotional code applied",
    "shipping_city": "Destination city for the shipment",
    "country": "Country of the shipping address",
    "total_price": "Total price before discount"
}

# Start table header
markdown_output = [
    "| Field | Type | Description | Source |",
    "|-------|------|-------------|--------|"
]

# Add entries from primary CSV
for column in df.columns:
    col_type = str(df[column].dtype)
    description = field_info.get(column, "No description available")
    markdown_output.append(f"| {column} | {col_type} | {description} | Primary |")

# Add product descriptions from secondary CSV
for _, product in product_catalog.iterrows():
    field_name = f"product: {product['product']}"
    markdown_output.append(
        f"| {field_name} | string | {product['description']} | Secondary (Product Catalog) |"
    )

# Join and print Markdown table
final_markdown = "\n".join(markdown_output)
print(final_markdown)

| Field | Type | Description | Source |
|-------|------|-------------|--------|
| date | object | Order date of the transaction | Primary |
| customer_id | object | Unique identifier for each customer or order | Primary |
| product | object | Name of the product sold | Primary |
| price | float64 | Unit price of the product | Primary |
| quantity | int64 | Number of units sold in the transaction | Primary |
| coupon_code | object | Discount or promotional code applied | Primary |
| shipping_city | object | Destination city for the shipment | Primary |
| country | object | Country of the shipping address | Primary |
| total_price | float64 | Total price before discount | Primary |
| product: Laptop | string | Portable computer with full OS | Secondary (Product Catalog) |
| product: Smartphone | string | Touchscreen mobile phone | Secondary (Product Catalog) |
| product: Headphones | string | Over-ear wireless audio device | Secondary (Product Catalog) |
| product: Keyboard | string | Me